# TCC-CBB 

Objetivo: Testar aplicação dos algoritmos PCA, SVM e XGBoost em Python

Data: 31/01/2020

Desenvolvido por: Thyago e Ricardo

Este script tem por objetivo visualizar resultados da aplicação dos algoritmos SVM e XGBoost na base de dados BigML. O primeiro passo consiste em instalar as bibliotecas necessárias.

In [ ]:
!pip install bigml

In [ ]:
!pip install xgboost

Importando as bibliotecas BigML (API para download do dataset), OS (para verificar o caminho que se está executando o python) e Pandas (para criação dos dataframes, estrutura de dados utilizada nos algoritmos)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
import bigml.api

In [ ]:
import os

In [ ]:
import pandas as pd

In [ ]:
from bigml.api import BigML

# Leitura da base de dados
Utilização das credenciais para leitura do dataset por meio da API:

In [ ]:
api = BigML('ricardomorellosantos','b34ec3c18161b1da38b0c5e04520224f7544405e')

Leitura do dataset e gravando-o no diretório atual em um arquivo CSV

In [ ]:
api.download_dataset(dataset='dataset/5e356cd41efc9271bf006ea2', filename=os.getcwd()+'\\bigml.csv')

Leitura do dataset em uma estrutura de dados DataFrame do pandas. Um dataframe é uma estrutura que permite armazenar dados heterogêneos de maneira tabular.

In [ ]:
df_bigml = pd.read_csv(os.getcwd()+'\\bigml.csv')

Apresentando informações sobre a base de dados já armazenada em um dataframe.

Primeiros 5 registros:

In [ ]:
df_bigml.head()

Informações sobre colunas e número de registros:

In [ ]:
df_bigml.info()

# Preparação dos dados para utilização do algoritmo PCA
Dados do tipo string serão convertidos para números para aplicação do PCA.
Colunas que possuem valores binários (Yes e No ou True e False) serão alteradas para Yes = 0, True = 0, No = 1 e False = 1, transformando estes valores em números.

Função para, dada uma palavra (Yes ou No, True ou False), retornar um valor booleano:

In [ ]:
def converte_binario(palavra):
    if palavra=='Yes' or palavra==True:
        return 0
    elif palavra=='No' or palavra==False:
        return 1

Aplicando essa função nas colunas Voice Mail Plan e International Plan (que contém valores Yes e No) e Churn (que contém True ou False). A coluna Churn é o target do dataframe. A coluna target é aquela que contém a resposta da classificação. Neste caso, se o usuário deixou ou manteve o plano com a operadora de telefonia.

In [ ]:
df_bigml['Voice mail plan'] = df_bigml['Voice mail plan'].apply(converte_binario)

In [ ]:
df_bigml['International plan'] = df_bigml['International plan'].apply(converte_binario)

In [ ]:
df_bigml['Churn'] = df_bigml['Churn'].apply(converte_binario)

Apreentando as primeiras 5 linhas do novo dataframe, para verificar se a transformação foi feita da maneira correta:

In [ ]:
df_bigml.head()

A coluna Churn é o target desta classificação. Portanto, ela será armazenada em um dataframe a parte, para que ela não interfira no algoritmo PCA e para realizar a separação do dataset.

In [ ]:
df_bigml_target = df_bigml['Churn']

Excluindo a coluna Churn do dataframe original:

In [ ]:
df_bigml.drop('Churn', axis=1, inplace=True)

Visualizando os 5 primeiros registros do novo dataframe que contém apenas o resultado da classificação, df_bigml_target:

In [ ]:
df_bigml_target.head()

O dataframe antigo contém agora apenas as colunas não-target, visto que a coluna Churn foi excluída e copiada para outro dataframe:

In [ ]:
df_bigml.head()

Estas linhas apenas foram executadas para verificar a proporção de Yes e No nas colunas após a transformação destes valores para 0 e 1. Nota-se que os valores 1 (No) e 0 (Yes) contém os mesmos números de registros de antes da transformação. Portanto, a função foi aplicada corretamente.

In [ ]:
df_bigml['Voice mail plan'].value_counts()

In [ ]:
states = df_bigml['State'].value_counts()

Agora, os dados de estados serão transformados em números para aplicação do PCA. Cada estado será armazenado em um vetor, e seu correspondente na coluna do dataframe assumirá o valor do índice deste vetor. Por exemplo, no vetor de estados v = ['KS', 'CA', 'WV', 'OH'], o estado da Califórnia assume o índice 1. Desta maneira, na coluna do dataframe, seu novo valor será substituído de Ca para 1.

In [ ]:
array_estados = []
i = 0
for index, val in states.iteritems():
    array_estados.append(index)    

In [ ]:
i = 0
for estado in array_estados:
    df_bigml['State'] = df_bigml['State'].replace(to_replace=estado, value=i)
    i = i+1

# Normalização dos dados para aplicação do PCA

Utilizamos um normalizador para manter os dados na mesma escala, evitando que diferenças em escalas nos dados interfiram negativamente no PCA. Importando a classe do normalizador:

In [ ]:
from sklearn.preprocessing import StandardScaler

Instanciando a classe:

In [ ]:
scaler = StandardScaler()

Ajustando o normalizador com o dataframe:

In [ ]:
scaler.fit(df_bigml)

Transformando os dados do dataframe para o mesmo espaço numérico:

In [ ]:
scaled_data = scaler.transform(df_bigml)

# Aplicação do PCA
Então, aplicou-se o PCA com 3 componentes. Importando a classe do PCA:

In [ ]:
from sklearn.decomposition import PCA

Instanciando a classe:

In [ ]:
pca = PCA(n_components = 3)

Ajustando o PCA com os dados. Nota-se que, agora, não estamos utilizando o dataframe original df_bigml, e sim o dataframe com os dados normalizados, denominado scaled_data.

In [ ]:
pca.fit(scaled_data)

Matriz que conterá os dados após a aplicação do pca:

In [ ]:
x_pca = pca.transform(scaled_data)

Visualizando as novas dimensões:

A matriz x_pca, que contém os dados após aplicação do PCA, possui apenas três colunas, referentes aos três componentes principais. Por outro lado, o dataframe scaled_data possui 19 dimensões, referentes às dimensões originais do dataset.

In [ ]:
x_pca.shape

In [ ]:
scaled_data.shape

Optamos por visualizar os dados do PCA num gráfico 3D interativo. Isto permite ver a distribuição espacial dos dados. 
Importando as bibliotecas para visualização 3D dos dados:

In [ ]:
import plotly.express as px
import cufflinks as cf

Algumas configurações da biblioteca plotly:

In [ ]:
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline

Criando um dataframe com o vetor de saída dos dados do PCA. Este dataframe será utilizado apenas para visualização:

In [ ]:
df_pca = pd.DataFrame(data=x_pca, columns = ['PC1','PC2','PC3'])

Configurando e apresentando o gráfico:

In [ ]:
fig = px.scatter_3d(df_pca, x='PC1', y='PC2', z='PC3',
                    color=df_bigml_target)
                    
fig.update_layout(
    title={
        'text': "Gráfico de dispersão PCA",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

# Divisão da base de dados em treino e teste

Para dividir a base de dados, optamos pelo fator 0.30. Importando a biblioteca para separação do dataframe:

In [ ]:
from sklearn.model_selection import train_test_split

Dividindo o dataframe em treino e teste. X_train e X_test contém os dados dos componentes, enquanto y_train e y_test contém os resultados da classificação.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_pca, df_bigml_target, test_size=0.30)

# Aplicação do SVM

Importando a biblioteca do SVM:

In [ ]:
from sklearn.svm import SVC

Instanciando uma classe do SVM:

In [ ]:
model = SVC(probability=True, gamma = 'auto')

Ajustando o SVM à divisão de treino: 

In [ ]:
model.fit(X_train, y_train)

Realizando a predição do SVM na divisão de teste:

In [ ]:
predicoes_svm = model.predict(X_test)

# Avaliação do SVM

Utilizaremo as bibliotecas classification_report e confusion_matrix para apresentar as métricas da avaliação da classificação do SVM. Importando as bibliotecas:

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Apresentação dos resultados da classificação do SVM:

In [ ]:
print(classification_report(y_test, predicoes_svm))

In [ ]:
print(confusion_matrix(y_test, predicoes_svm))

# Aplicação do XGBoost
Importando as bibliotecas do XGBoost. Esta biblioteca não é padrão do Python, e deve ser instalada.

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

Instanciando o classificador XGBoost:

In [ ]:
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10, probability = True)

Ajustando o classificador e realizando a predição:

In [ ]:
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

# Avaliação do XGBoost
Apresentação dos resultados para o algoritmo XGBoost:

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
print(confusion_matrix(y_test, preds))

In [ ]:
import seaborn as sns

# Média e desvio padrão do SVM

In [ ]:
media_anterior = 0
auc_svm = []
medias_svm = []
convergencia = False
qtd_iguais = 0
pca = PCA(n_components = 16)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)
while(not convergencia):
    X_train, X_test, y_train, y_test = train_test_split(x_pca, df_bigml_target, test_size=0.20)
    model.fit(X_train,y_train)
    predictedprob = model.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, predictedprob[:,1])
    roc_auc = auc(fpr, tpr)
    auc_svm.append(roc_auc)
    media_aucs = round(sum(auc_svm) / len(auc_svm), 2) 
    medias_svm.append(media_aucs)
    print(media_aucs)
    if media_anterior == media_aucs:
        qtd_iguais = qtd_iguais + 1
    else:
        qtd_iguais = 0
    if qtd_iguais == 10:
        convergencia = True
    media_anterior = media_aucs


In [ ]:
import statistics
fig = plt.figure(figsize=(12,6))
data_svm = pd.DataFrame({'AUC':auc_svm, 'Média AUCs':medias_svm})
ax = data_svm.plot(title = 'AUC X Iteração SVM', figsize = (12, 6))
ax.text(4.5, 0.84, 'Desvio padrão:{}'.format(round(statistics.stdev(medias_svm), 4)))
plt.savefig('auc_iteracao_svm_bigml.PNG')
print(sorted(auc_svm))

# Média e desvio padrão do XGBoost

In [ ]:
media_anterior = 0
auc_xgboost = []
medias_xgboost = []
convergencia = False
qtd_iguais = 0
pca = PCA(n_components = 17)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)
while(not convergencia):
    X_train, X_test, y_train, y_test = train_test_split(x_pca, df_bigml_target, test_size=0.10)
    xg_reg.fit(X_train,y_train)
    predictedprob = xg_reg.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, predictedprob[:,1])
    roc_auc = auc(fpr, tpr)
    auc_xgboost.append(roc_auc)
    media_aucs = round(sum(auc_xgboost) / len(auc_xgboost), 2) 
    medias_xgboost.append(media_aucs)
    print(roc_auc)
    if media_anterior == media_aucs:
        qtd_iguais = qtd_iguais + 1
    else:
        qtd_iguais = 0
    if qtd_iguais >= 10:
        convergencia = True
    media_anterior = media_aucs


In [ ]:
import statistics
fig = plt.figure(figsize=(12,6))
data_xgboost = pd.DataFrame({'AUC':auc_xgboost, 'Média AUCs':medias_xgboost})
ax = data_xgboost.plot(title = 'AUC X Iteração XGBoost', figsize = (12, 6))
ax.text(11, 0.7, 'Desvio padrão:{}'.format(round(statistics.stdev(medias_xgboost), 4)))
plt.savefig('auc_iteracao_xgboost_bigml.PNG')
print(medias_xgboost)

# Gráfico de média e desvio padrão xgboost e svm

In [ ]:
df_auc_med_desvpad = pd.DataFrame(auc_med_desvpad_xgboost)
print(len(auc_med_desvpad_xgboost))
print(len(auc_med_desvpad_svm))


fig = plt.figure(figsize = (12,6))
plt.ylabel('AUC')

g = sns.lineplot(data = df_auc_med_desvpad_svm, marker = 'o', )
g.legend('')

In [ ]:
df_auc_med_desvpad['SVM'] = auc_med_desvpad_svm
df_auc_med_desvpad

# Curva ROC SVM

In [ ]:
pca = PCA(n_components = 3)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)
X_train, X_test, y_train, y_test = train_test_split(x_pca, df_bigml_target, test_size=0.30)
model.fit(X_train,y_train)
predictedprobSVC = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, predictedprobSVC[:,1])
roc_auc = auc(fpr, tpr)

In [ ]:
#GRAPH DATA
fig = plt.figure()
plt.xlabel('1-Especificidade')
plt.ylabel('Sensibilidade')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.title('Curva ROC SVM')
plt.plot(fpr, tpr, color='blue', lw=2, label='SVM ROC area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.show()
fig.savefig('ROC_svm.PNG')

# Curva ROC XGBoost

In [ ]:
#for i in range(1,20):
pca = PCA(n_components = 16)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)
X_train, X_test, y_train, y_test = train_test_split(x_pca, df_bigml_target, test_size=0.20)
model.fit(X_train,y_train)
predictedprob = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, predictedprob[:,1])
roc_auc = auc(fpr, tpr)
print(roc_auc)

In [ ]:
#GRAPH DATA
fig = plt.figure()
plt.xlabel('1-Especificidade')
plt.ylabel('Sensibilidade')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.title('Curva ROC SVM')
plt.plot(fpr, tpr, color='blue', lw=2, label='SVM ROC area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.show()
fig.savefig('ROC_SVM_otimizado.PNG')

# Avaliação do PCA e divisão teste/treino - SVM

In [ ]:
auc_total = []
auc_divisao = []
convergencia = False

numAucsComponente = []
numAucsDivisao = []
for divisao in np.arange(0.1, 1, 0.1):
    print(divisao)
    for numComponentes in range(1,20):
        print(numComponentes)
        auc_componente = []
        media_anterior = 0
        qtd_iguais = 0
        pca = PCA(n_components = numComponentes)
        pca.fit(scaled_data)
        x_pca = pca.transform(scaled_data)
        convergencia = False
        while(not convergencia):
            X_train, X_test, y_train, y_test = train_test_split(x_pca, df_bigml_target, test_size=divisao)
            model.fit(X_train,y_train)
            predictedprob = model.predict_proba(X_test)
            fpr, tpr, thresholds = roc_curve(y_test, predictedprob[:,1])
            roc_auc = auc(fpr, tpr)
            auc_componente.append(roc_auc)
            media_aucs = round(sum(auc_componente) / len(auc_componente), 2) 
            #print(media_aucs)
            if media_anterior == media_aucs:
                qtd_iguais = qtd_iguais + 1
            else:
                qtd_iguais = 0
            if qtd_iguais >= 10:
                numAucsComponente.append(len(auc_componente))
                auc_total.append(media_aucs)
                convergencia = True
            media_anterior = media_aucs
    numAucsDivisao.append(numAucsComponente)
    numAucsComponente = []
    auc_divisao.append(auc_total)
    auc_total = []
    

### Gráfico de assertividade com base no número de componentes do SVM

In [ ]:
def add(x):
    return x+1

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (12,6))

df_divisao = pd.DataFrame(auc_divisao)
df_divisao.set_index(np.arange(1,10), inplace = True)
df_divisao.rename(add, axis='columns', inplace=True)
sns.heatmap(data = df_divisao)
plt.title("Assertividade SVM")
plt.savefig('assertividade-svm-divisao-numcomponentes.PNG')
plt.rcParams.update({'font.size': 20})

### Gráfico de mapa de calor do SVM

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (12,6))

df = pd.DataFrame(numAucsDivisao)
df.set_index(np.arange(1,10), inplace = True)
df.rename(add, axis='columns', inplace=True)
sns.heatmap(data = df, cmap = 'YlGnBu')
plt.title("Quantidade de iterações até a convergência SVM")
plt.savefig('iteracoes-svm-divisao-numcomponentes.PNG')

# Avaliação do PCA e divisão teste/treino - XGBoost

In [ ]:
auc_total = []
auc_divisao = []
convergencia = False

numAucsComponente = []
numAucsDivisao = []
for divisao in np.arange(0.1, 1, 0.1):
    print(divisao)
    for numComponentes in range(1,20):
        print(numComponentes)
        auc_componente = []
        media_anterior = 0
        qtd_iguais = 0
        pca = PCA(n_components = numComponentes)
        pca.fit(scaled_data)
        x_pca = pca.transform(scaled_data)
        convergencia = False
        while(not convergencia):
            X_train, X_test, y_train, y_test = train_test_split(x_pca, df_bigml_target, test_size=divisao)
            xg_reg.fit(X_train,y_train)
            predictedprob = xg_reg.predict_proba(X_test)
            fpr, tpr, thresholds = roc_curve(y_test, predictedprob[:,1])
            roc_auc = auc(fpr, tpr)
            auc_componente.append(roc_auc)
            media_aucs = round(sum(auc_componente) / len(auc_componente), 2) 
            print(len(auc_componente))
            if media_anterior == media_aucs:
                qtd_iguais = qtd_iguais + 1
            else:
                qtd_iguais = 0
            if qtd_iguais >= 20:
                numAucsComponente.append(len(auc_componente))
                auc_total.append(media_aucs)
                print(len(auc_componente))
                convergencia = True
            media_anterior = media_aucs
    numAucsDivisao.append(numAucsComponente)
    numAucsComponente = []
    auc_divisao.append(auc_total)
    auc_total = []
    

### Gráfico de assertividade com base no número de componentes do XGBoost

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (12,6))

df_divisao = pd.DataFrame(auc_divisao)
df_divisao.set_index(np.arange(1,10), inplace = True)
df_divisao.rename(add, axis='columns', inplace=True)
sns.heatmap(data = df_divisao)
plt.title("Assertividade XGBoost")
plt.savefig('assertividade-xgboost-divisao-numcomponentes.PNG')

### Gráfico de mapa de calor para o XGBoost

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (12,6))

df = pd.DataFrame(numAucsDivisao)
df.set_index(np.arange(1,10), inplace = True)
df.rename(add, axis='columns', inplace=True)
sns.heatmap(data = df, cmap = 'YlGnBu')
plt.title("Quantidade de iterações até a convergência XGBoost")
plt.savefig('iteracoes-xgboost-divisao-numcomponentes.PNG')